In [1]:
# imports
import sys
import os
import socket
import pandas as pd
import glob
from datetime import datetime
from functools import reduce

pd.set_option('display.max_columns', None)

In [2]:
# function to format dates
def formatdate(data):
    data['sysdate'] = data['sysdate'].str.replace('/', '')
    data['sysdate']= pd.to_datetime(data['sysdate'], format='%d%m%Y')
    data['eventdate']= data['eventdate'].str.replace('/', '')
    data['eventdate'] = pd.to_datetime(data['eventdate'], format='%d%m%Y')
    data['eventdate'].fillna(data['sysdate'], inplace = True)
    return data

# function to get latest record in a df of e_patids and dates
def getlatest(data):
    data = data[['e_patid', 'eventdate']].groupby(['e_patid']).max()
    return data

In [3]:
next(os.walk('/mnt/lustre/users/maths/tws21/seb_tmp/bsms2730/bsms2730'))

('/mnt/lustre/users/maths/tws21/seb_tmp/bsms2730/bsms2730',
 ['Dictionary Browsers',
  'Marco Antipsychotics Files',
  'Lookups',
  'Linked Data',
  'Primary Care',
  'Documentation'],
 ['19_050.zip'])

In [4]:
next(os.walk('/mnt/lustre/users/maths/tws21/seb_tmp/bsms2730/bsms2730/Primary Care/CPRD GOLD'))

('/mnt/lustre/users/maths/tws21/seb_tmp/bsms2730/bsms2730/Primary Care/CPRD GOLD',
 [],
 ['19_050_gold_extract_therapy_072.txt',
  '19_050_gold_extract_test_010.txt',
  '19_050_gold_extract_therapy_055.txt',
  '19_050_gold_extract_therapy_113.txt',
  '19_050_gold_extract_therapy_033.txt',
  '19_050_gold_extract_therapy_077.txt',
  '19_050_gold_extract_therapy_023.txt',
  '19_050_gold_extract_clinical_028.txt',
  '19_050_gold_extract_test_018.txt',
  '19_050_gold_extract_test_027.txt',
  '19_050_gold_extract_clinical_010.txt',
  '19_050_gold_extract_test_012.txt',
  '19_050_gold_extract_additional_001.txt',
  '19_050_gold_extract_test_023.txt',
  '19_050_gold_extract_therapy_036.txt',
  '19_050_gold_extract_therapy_062.txt',
  '19_050_gold_extract_clinical_024.txt',
  '19_050_gold_extract_therapy_007.txt',
  '19_050_gold_extract_therapy_079.txt',
  '19_050_gold_extract_therapy_060.txt',
  '19_050_gold_extract_therapy_099.txt',
  '19_050_gold_extract_therapy_061.txt',
  '19_050_gold_extr

In [5]:
path = '/mnt/lustre/users/maths/tws21/seb_tmp/bsms2730/bsms2730/Primary Care/CPRD GOLD/'

In [6]:
# read in cohort file
cohort = pd.read_parquet(os.getcwd() + '/late_by_symptom_date_cohort_file.parquet')

In [7]:
cohort.head()

,e_patid,first_diagnosis_date,first_diagnosed_in_primary_care,gender,age_at_dx,e_pracid,region,uts,e2015_imd_5,dx_minus6m,dx_minus12m,dx_minus18m,dx_minus24m,dx_minus3y,dx_minus5y,late_dx
0,24117254,2005-10-04,1.0,1.0,70.0,54.0,10.0,2000-01-12,4.0,2005-04-04,2004-10-04,2004-04-04,2003-10-05,2002-10-05,2000-10-04,0.0
1,519045131,2004-02-11,0.0,1.0,74.0,131.0,2.0,1992-12-21,5.0,2003-08-12,2003-02-11,2002-08-12,2002-02-11,2001-02-11,1999-02-11,1.0
3,734003216,2009-08-26,1.0,2.0,93.0,216.0,10.0,2003-02-14,4.0,2009-02-24,2008-08-26,2008-02-25,2007-08-27,2006-08-27,2004-08-26,0.0
7,10485798,2014-01-02,0.0,1.0,89.0,98.0,6.0,1990-01-03,4.0,2013-07-03,2013-01-02,2012-07-03,2012-01-03,2011-01-03,2009-01-02,1.0
8,630194220,2003-05-15,0.0,2.0,74.0,220.0,7.0,1997-09-23,5.0,2002-11-13,2002-05-15,2001-11-13,2001-05-15,2000-05-15,1998-05-15,1.0


In [8]:
cohort.age_at_dx.describe()

count    25453.000000
mean        79.239932
std          8.576975
min         42.000000
25%         74.000000
50%         80.000000
75%         85.000000
max        108.000000
Name: age_at_dx, dtype: float64

In [9]:
# get unique e_patids
cohort_e_patids = list(cohort['e_patid'])

In [10]:
clinical_files = glob.glob(path + "*clinical*.txt") # list all files in the directory with "clinical" in filename

#read  all the clinical files in for cohort patients and add to a list
clinical_list = []
for filename in clinical_files:    
    df = pd.read_csv(filename, sep="\t")
    df = df[df['e_patid'].isin(cohort_e_patids)]
    df = formatdate(df)
    clinical_list.append(df)

clinical_merged = pd.concat(clinical_list, axis=0) # merge all the clinical files together

In [11]:
#read in and merge the referral files similarly

referral_files = glob.glob(path + "*referral*.txt")

referral_li = []
for filename in referral_files:
    df = pd.read_csv(filename, sep="\t")
    df = df[df['e_patid'].isin(cohort_e_patids)]
    df = formatdate(df)
    referral_li.append(df)

referral_merged = pd.concat(referral_li, axis=0)

In [12]:
#read in and merge all the test files similarly

test_files = glob.glob(path + "*test*.txt")

test_list = []
for filename in test_files:
    df = pd.read_csv(filename, sep="\t", dtype = ({'data8':'str'}))
    df = df[df['e_patid'].isin(cohort_e_patids)]
    df = formatdate(df)
    test_list.append(df)

test_merged = pd.concat(test_list, axis=0)

In [13]:
#read in and merge all the additional files in the same way but don't format for date as they link to clinical

additional_files = glob.glob(path + "*additional*.txt")

additional_list = []
for filename in additional_files:
    df = pd.read_csv(filename, sep="\t")
    df = df[df['e_patid'].isin(cohort_e_patids)]
    additional_list.append(df)

additional_merged = pd.concat(additional_list, axis=0)

In [14]:
# read in dementia codes for filtering the depression codes as some cross over
dementia_codes = pd.read_excel('/mnt/maths/tws21/dementia_codes.xlsx')
dementia_codes.head()

,medcode,readcode,readterm,type,probable,possible,diagnosis,history,admin,assessment,suspected,referral,drugs
0,1917,F110.00,Alzheimer's disease,Alzheimer's,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,7664,Eu00.00,[X]Dementia in Alzheimer's disease,Alzheimer's,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2,8195,Eu00z11,[X]Alzheimer's dementia unspec,Alzheimer's,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
3,11379,Eu00112,"[X]Senile dementia,Alzheimer's type",Alzheimer's,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
4,16797,F110000,Alzheimer's disease with early onset,Alzheimer's,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# read in merged ethnicity file
ethnicity = pd.read_parquet(os.getcwd() + '/Merged_Ethnicities.parquet')
ethnicity.head()

,e_patid,Ethnicity
0,24117254,White
1,519045131,White
2,734003216,White
3,10485798,White
4,630194220,White


In [16]:
# read in weight med codes
weight_codes = pd.read_excel(os.getcwd() + '/Risk Factor Codes/weight.dta.xls')
weight_codes.head()

,medcode,readcode,readterm,category
0,912,R030.00,[D]Anorexia,Underweight
1,32914,22K3.00,Body Mass Index low K/M2,Underweight
2,2135,E271.00,Anorexia nervosa,Underweight
3,7608,1612,Appetite loss - anorexia,Underweight
4,126,22A6.00,O/E - Underweight,Underweight


In [17]:
# read in alcohol use med codes
alcohol_codes = pd.read_csv(os.getcwd() + '/Risk Factor Codes/Comorbidity Codes/alcohol to read.csv')
alcohol_codes.head()

,medcode,readcode,readterm,consumptionlevel,heavydrinking,drinkingstatus
0,27,136..00,alcohol consumption,NaN,NaN,current
1,322,1364.00,Moderate drinker - 3-6u/day,moderate,NaN,current
2,385,1362.11,Drinks rarely,low,NaN,current
3,669,E250000,"Nondependent alcohol abuse, unspecified",high,yes,current
4,749,1362.12,Drinks occasionally,low,NaN,current


In [18]:
# read in anxiety use med codes
anxiety_codes = pd.read_csv(os.getcwd() + '/Risk Factor Codes/Comorbidity Codes/anxiety to read.csv')
anxiety_codes.head()

,readcode,medcode,clinicalevents,immunisationevents,referralevents,testevents,readterm,databasebuild,description
0,2258,13124,NaN,NaN,NaN,NaN,NaN,NaN,O/E - anxious
1,225J.00,19000,559.0,0.0,8.0,0.0,O/E - panic attack,Feb-09,O/E - panic attack
2,8G52.00,63521,4.0,0.0,0.0,0.0,Antiphobic therapy,Feb-09,Antiphobic therapy
3,8G94.00,9125,7665.0,0.0,331.0,0.0,Anxiety management training,Feb-09,Anxiety management training
4,8HHp.00,28925,881.0,0.0,628.0,0.0,Referral for guided self-help for anxiety,Feb-09,Referral for guided self-help for anxiety


In [19]:
# read in depression med codes
depression_codes = pd.read_csv(os.getcwd() + '/Risk Factor Codes/Comorbidity Codes/depression to read.csv')
depression_codes.head()

,readcode,medcode,clinicalevents,immunisationevents,referralevents,testevents,readterm,databasebuild,description
0,1465,2716,NaN,NaN,NaN,NaN,NaN,NaN,H/O: depression
1,1B17.00,1996,487649.0,0.0,16936.0,0.0,Depressed,Feb-09,Depressed
2,1B17.00,1996,487649.0,0.0,16936.0,0.0,Depressed,Feb-09,Depressed
3,1B17.11,4824,341299.0,0.0,14047.0,0.0,C/O - feeling depressed,Feb-09,C/O - feeling depressed
4,1B1U.00,9796,249326.0,0.0,10738.0,0.0,Symptoms of depression,Feb-09,Symptoms of depression


In [20]:
# read in serious mental illness med codes
smi_codes = pd.read_csv(os.getcwd() + '/Risk Factor Codes/Comorbidity Codes/SMI_plus_medcodes.csv')
smi_codes.head()

,readcode,medcode,clinicalevents,immunisationevents,referralevents,testevents,readterm,databasebuild,description
0,E100.00,32222,1130,0,22,0,Simple schizophrenia,Feb-09,Simple schizophrenia
1,E100.11,73295,5,0,0,0,Schizophrenia simplex,Feb-09,Schizophrenia simplex
2,E100000,15733,3075,0,107,0,Unspecified schizophrenia,Feb-09,Unspecified schizophrenia
3,E100100,23616,291,0,0,0,Subchronic schizophrenia,Feb-09,Subchronic schizophrenia
4,E100200,3984,4245,0,243,0,Chronic schizophrenic,Feb-09,Chronic schizophrenic


In [21]:
# read in epilepsy med codes
epilepsy_codes = pd.read_csv(os.getcwd() + '/Risk Factor Codes/Comorbidity Codes/epilepsy to read.csv')
epilepsy_codes.head()

,readcode,medcode,clinicalevents,immunisationevents,referralevents,testevents,readterm,databasebuild,description
0,1B1W.00,38919.0,101.0,0.0,1.0,0.0,Transient epileptic amnesia,Feb-09,Transient epileptic amnesia
1,2126000,8385.0,9443.0,0.0,40.0,0.0,Epilepsy resolved,Feb-09,Epilepsy resolved
2,212J.00,12848.0,5413.0,0.0,9.0,0.0,Epilepsy resolved,Feb-09,Epilepsy resolved
3,667..00,6983.0,150039.0,0.0,2053.0,1.0,Epilepsy monitoring,Feb-09,Epilepsy monitoring
4,667A.00,20566.0,777.0,0.0,1.0,0.0,Epilepsy treatment stopped,Feb-09,Epilepsy treatment stopped


In [22]:
# read in asthma med codes
asthma_codes = pd.read_csv(os.getcwd() + '/Risk Factor Codes/Comorbidity Codes/asthma_to_read.csv')
asthma_codes.head()

,readcode,medcode,clinicalevents,immunisationevents,referralevents,testevents,readterm,databasebuild,description
0,173A.00,5867.0,54394.0,0.0,186.0,3.0,Exercise induced asthma,February 2009,Exercise induced asthma
1,173c.00,22752.0,717.0,0.0,37.0,0.0,Occupational asthma,February 2009,Occupational asthma
2,173d.00,73522.0,221.0,0.0,1.0,0.0,Work aggravated asthma,February 2009,Work aggravated asthma
3,1780,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aspirin induced asthma
4,1O2..00,11370.0,19157.0,0.0,48.0,0.0,Asthma confirmed,February 2009,Asthma confirmed


In [23]:
# read chronic fatigue syndrome med codes
cfs_codes = pd.read_csv(os.getcwd() + '/Risk Factor Codes/Comorbidity Codes/CFS to read_with_extra.csv')
cfs_codes.head()

,medcode,clinicalevents,referralevents,testevents,immunisationevents,readcode,readterm
0,4546,32056,1839,0,0,F286.00,Chronic fatigue syndrome
1,1042,31394,1620,4,0,R007400,[D]Postviral (asthenic) syndrome
2,7529,22491,1918,0,0,F286.11,CFS - Chronic fatigue syndrome
3,6190,11608,230,0,0,F286.12,Postviral fatigue syndrome
4,1040,5645,333,0,0,F286.15,Myalgic encephalomyelitis


In [24]:
# read in chronic heart disease med codes
chd_codes = pd.read_csv(os.getcwd() + '/Risk Factor Codes/Comorbidity Codes/CHD_to_read.csv')
chd_codes.head()

,readcode,medcode,clinicalevents,immunisationevents,referralevents,testevents,readterm,databasebuild,description
0,G3...00,240,677565,0,11334,1647,Ischaemic heart disease,February 2009,Ischaemic heart disease
1,G3...11,24783,925,0,1,0,Arteriosclerotic heart disease,February 2009,Arteriosclerotic heart disease
2,G3...12,20416,2393,0,10,0,Atherosclerotic heart disease,February 2009,Atherosclerotic heart disease
3,G3...13,1792,161167,0,3154,10,IHD - Ischaemic heart disease,February 2009,IHD - Ischaemic heart disease
4,G31..00,27951,226,0,5,0,Other acute and subacute ischaemic heart disease,February 2009,Other acute and subacute ischaemic heart disease


In [25]:
# read in chronic kidney disease med codes
ckd_codes = pd.read_csv(os.getcwd() + '/Risk Factor Codes/Comorbidity Codes/CKD to read.csv')
ckd_codes.head()

,medcode,coding_system,description,entity,list_name,upload_date
0,10081,Read,Chronic uraemia,diagnostic,137 - Chronic Kidney Disease,24/06/2014
1,10647,Read,Nephritis - chronic,diagnostic,137 - Chronic Kidney Disease,24/06/2014
2,10809,Read,Chronic membranous glomerulonephritis,diagnostic,137 - Chronic Kidney Disease,24/06/2014
3,11553,Read,Kidney transplant failure and rejection,diagnostic,137 - Chronic Kidney Disease,24/06/2014
4,11875,Read,Nephropathy - chronic,diagnostic,137 - Chronic Kidney Disease,24/06/2014


In [26]:
# read in alcohol audit and early screening med codes
alc_audit_codes = pd.read_csv(os.getcwd() + '/Risk Factor Codes/Comorbidity Codes/Code lists for audit or fast screening.csv')
alc_audit_codes.head()

,medcode,readcode,readterm,Type
0,32850,ZR1F.00,alcohol use disorders identification test,AUDIT
1,59873,ZR1F.11,audit - alcohol use disorders identification test,AUDIT
2,90714,9k15.00,alcohol screen - audit completed,AUDIT
3,93624,388u.00,fast alcohol screening test,FAST
4,94485,9k18.00,alcohol screen - audit pc completed,AUDIT-PC


In [27]:
# read in chronic obstructive pulmonary disease med codes
copd_codes = pd.read_csv(os.getcwd() + '/Risk Factor Codes/Comorbidity Codes/COPD to read.csv')
copd_codes.head()

,readcode,medcode,clinicalevents,immunisationevents,referralevents,testevents,readterm,databasebuild,description
0,H3...00,1001,481569,0,12259,148,Chronic obstructive pulmonary disease,Feb-09,Chronic obstructive pulmonary disease
1,H31..00,3243,32135,0,731,31,Chronic bronchitis,Feb-09,Chronic bronchitis
2,H310.00,25603,958,0,0,0,Simple chronic bronchitis,Feb-09,Simple chronic bronchitis
3,H310000,15626,1738,0,46,0,Chronic catarrhal bronchitis,Feb-09,Chronic catarrhal bronchitis
4,H310z00,61118,58,0,0,0,Simple chronic bronchitis NOS,Feb-09,Simple chronic bronchitis NOS


In [28]:
# read in heart failure med codes
heart_failure_codes = pd.read_csv(os.getcwd() + '/Risk Factor Codes/Comorbidity Codes/heart failure to read.csv')
heart_failure_codes.head()

,readcode,medcode,clinicalevents,immunisationevents,referralevents,testevents,readterm,databasebuild,description
0,1O1..00,9913,9229,0,702,0,Heart failure confirmed,Feb-09,Heart failure confirmed
1,G1yz100,22262,170,0,13,0,Rheumatic left ventricular failure,Feb-09,Rheumatic left ventricular failure
2,G232.00,21837,35,0,4,0,Hypertensive heart&renal dis wth (congestive) ...,Feb-09,Hypertensive heart&renal dis wth (congestive) ...
3,G234.00,57987,8,0,1,0,Hyperten heart&renal dis+both(congestv)heart a...,Feb-09,Hyperten heart&renal dis+both(congestv)heart a...
4,G58..00,2062,123236,0,8309,19,Heart failure,Feb-09,Heart failure


In [29]:
# read in hypertension med codes
hypertension_codes = pd.read_csv(os.getcwd() + '/Risk Factor Codes/Comorbidity Codes/hypertension_to_read.csv')
hypertension_codes.head()

,readcode,medcode,clinicalevents,immunisationevents,referralevents,testevents,readterm,databasebuild,description
0,2126100,3269.0,25005.0,0.0,44.0,0.0,Hypertension resolved,Feb-09,Hypertension resolved
1,2126100,3269.0,25005.0,0.0,44.0,0.0,Hypertension resolved,Feb-09,Hypertension resolved
2,212K.00,19342.0,9816.0,0.0,1.0,0.0,Hypertension resolved,Feb-09,Hypertension resolved
3,212K.00,19342.0,9816.0,0.0,1.0,0.0,Hypertension resolved,Feb-09,Hypertension resolved
4,662..12,4444.0,3217898.0,0.0,16947.0,110.0,Hypertension monitoring,Feb-09,Hypertension monitoring


In [30]:
# read in osteoporosis med codes
osteoporosis_codes = pd.read_csv(os.getcwd() + '/Risk Factor Codes/Comorbidity Codes/osteoporosis to read.csv')
osteoporosis_codes.head()

,readcode,medcode,clinicalevents,immunisationevents,referralevents,testevents,readterm,databasebuild,coding_system,description,entity,list_name,upload_date
0,585O.00,51891,2,0,2,29,Quantitative ultrasound scan of heel - result ...,Feb-09,Read,Quantitative ultrasound scan of heel - result ...,diagnostic,34 - Osteoporosis,24/06/2014
1,58E4.00,13987,78,0,15,1182,Forearm DXA scan result osteoporotic,Feb-09,Read,Forearm DXA scan result osteoporotic,diagnostic,34 - Osteoporosis,24/06/2014
2,58E8.00,37972,15,0,1,729,Heel DXA scan T score,Feb-09,Read,Heel DXA scan T score,diagnostic,34 - Osteoporosis,07/11/2013
3,58EA.00,46510,25,0,6,643,Heel DXA scan result osteoporotic,Feb-09,Read,Heel DXA scan result osteoporotic,diagnostic,34 - Osteoporosis,07/11/2013
4,58EA.00,46510,25,0,6,643,Heel DXA scan result osteoporotic,Feb-09,Read,Heel DXA scan result osteoporotic,diagnostic,34 - Osteoporosis,24/06/2014


In [31]:
cohort2 = cohort.merge(right = ethnicity, on = ['e_patid'], how = 'left')
cohort2 = cohort2[cohort2.Ethnicity != 'Unspecified']
cohort2.head()

,e_patid,first_diagnosis_date,first_diagnosed_in_primary_care,gender,age_at_dx,e_pracid,region,uts,e2015_imd_5,dx_minus6m,dx_minus12m,dx_minus18m,dx_minus24m,dx_minus3y,dx_minus5y,late_dx,Ethnicity
0,24117254,2005-10-04,1.0,1.0,70.0,54.0,10.0,2000-01-12,4.0,2005-04-04,2004-10-04,2004-04-04,2003-10-05,2002-10-05,2000-10-04,0.0,White
1,519045131,2004-02-11,0.0,1.0,74.0,131.0,2.0,1992-12-21,5.0,2003-08-12,2003-02-11,2002-08-12,2002-02-11,2001-02-11,1999-02-11,1.0,White
2,734003216,2009-08-26,1.0,2.0,93.0,216.0,10.0,2003-02-14,4.0,2009-02-24,2008-08-26,2008-02-25,2007-08-27,2006-08-27,2004-08-26,0.0,White
3,10485798,2014-01-02,0.0,1.0,89.0,98.0,6.0,1990-01-03,4.0,2013-07-03,2013-01-02,2012-07-03,2012-01-03,2011-01-03,2009-01-02,1.0,White
4,630194220,2003-05-15,0.0,2.0,74.0,220.0,7.0,1997-09-23,5.0,2002-11-13,2002-05-15,2001-11-13,2001-05-15,2000-05-15,1998-05-15,1.0,White


In [32]:
#select rows wtih codes for drinking and merge in the drinking status and heavy drinking variables
clin_drink = clinical_merged[clinical_merged.medcode.isin(alcohol_codes.medcode)][['e_patid', 'eventdate', 'medcode']]
ref_drink = referral_merged[referral_merged.medcode.isin(alcohol_codes.medcode)][['e_patid', 'eventdate', 'medcode']]
clin_drink = pd.concat([clin_drink, ref_drink])
clin_drink = pd.merge(clin_drink, alcohol_codes[['medcode', 'heavydrinking', 'drinkingstatus']], how="left", on='medcode')
#select only data at least 6 months before diagnosis
clin_drink = pd.merge(clin_drink, cohort[['e_patid', 'dx_minus6m']], how="left", on="e_patid").copy(deep=True)
clin_drink = clin_drink[clin_drink["eventdate"] < clin_drink["dx_minus6m"]].copy(deep=True)
#split by drinking status and heavy drinking
curr_drink_clin = clin_drink[clin_drink.drinkingstatus=="current"][['e_patid', 'eventdate']].copy(deep=True)
never_drink_clin = clin_drink[clin_drink.drinkingstatus=="never"][['e_patid', 'eventdate']].copy(deep=True)
former_drink_clin = clin_drink[clin_drink.drinkingstatus=="former"][['e_patid', 'eventdate']].copy(deep=True)
heavy_drink_clin =  clin_drink[clin_drink.heavydrinking=="yes"][['e_patid', 'eventdate']].copy(deep=True)

ent5 = additional_merged[additional_merged.enttype==5].copy(deep=True)
#data1 is status (0 = data not entered, 1 = Yes, 2=No, 3=Ex, data2 is units)
ent5['data2'] = ent5['data2'].astype(float)

ent5 = pd.merge(ent5, clinical_merged[['e_patid', 'adid', 'eventdate']], 
                                   on = ['e_patid', 'adid'], how = 'left').copy(deep=True)
ent5.drop(['data5', 'data6', 'data7'], axis=1, inplace=True)
#select only data at least 6 months before diagnosis
ent5 = pd.merge(ent5, cohort[['e_patid', 'dx_minus6m']], how="left", on="e_patid").copy(deep=True)
ent5 = ent5[ent5["eventdate"] < ent5["dx_minus6m"]].copy(deep=True)
curr_drinker = ent5[ent5.data1=="1"].copy(deep=True)
heavy_drinker = curr_drinker[curr_drinker.data2>14].copy(deep=True) #heavy drinker if consumption more than 14 units
non_drinker =  ent5[ent5.data1=="2"].copy(deep=True)
non_drinker = non_drinker.loc[~(non_drinker['data2']>0)] #get rid of rows in non-drinker with a value>0 for units per week
ex_drinker = ent5[ent5.data1=="3"].copy(deep=True)

alc_audit_codes_clin = clinical_merged[clinical_merged.medcode.isin(alc_audit_codes.medcode)]
#NB quite a lot of these codes are associated with entity type 2 which has 0 data fields so we can't assess the score
#quite a lot of the others are associated with entity 5 which is units per week rather tha audit/fast score
#select out the rows where enttype = 372 for all these files as this means a score is available and onoy keep relevant columns
alc_audit_codes_clin = alc_audit_codes_clin[alc_audit_codes_clin.enttype==372][['e_patid', 'medcode', 'eventdate', 'enttype', 'adid']]
#select only data at least 6 months before diagnosis
alc_audit_codes_clin = pd.merge(alc_audit_codes_clin, cohort[['e_patid', 'dx_minus6m']], how="left", on="e_patid").copy(deep=True)
alc_audit_codes_clin = alc_audit_codes_clin[alc_audit_codes_clin["eventdate"] < alc_audit_codes_clin["dx_minus6m"]].copy(deep=True)
#merge in the datafields from the additional file
#for ent372 data1 = result, data2 = condition (medcode), data3 = scoring method data4 =m qualifier 1=negative, 2=positive
alc_audit_codes_clin = pd.merge(alc_audit_codes_clin, additional_merged[['e_patid', 'adid', 'data1', 'data2', 'data3', 'data4']], 
                                   on = ['e_patid', 'adid'], how = 'left').copy(deep=True)
#change data1 to float format as currently string
alc_audit_codes_clin['data1'] = alc_audit_codes_clin['data1'].astype(float)
#split by the different screening tests and create heavy drinking subset
audit_clin = alc_audit_codes_clin[alc_audit_codes_clin.medcode.isin(alc_audit_codes.medcode[alc_audit_codes.Type=="AUDIT"])]
audit_c_clin = alc_audit_codes_clin[alc_audit_codes_clin.medcode.isin(alc_audit_codes.medcode[alc_audit_codes.Type=="AUDIT-C"])]
audit_pc_clin = alc_audit_codes_clin[alc_audit_codes_clin.medcode.isin(alc_audit_codes.medcode[alc_audit_codes.Type=="AUDIT-PC"])]
fast_clin = alc_audit_codes_clin[alc_audit_codes_clin.medcode.isin(alc_audit_codes.medcode[alc_audit_codes.Type=="FAST"])]
#select the scores indicating heavy drinking for each test type and merge
heavy372 = pd.concat([audit_clin[audit_clin.data1>7], audit_c_clin[audit_c_clin.data1>4], audit_pc_clin[audit_pc_clin.data1>4], fast_clin[fast_clin.data1>1]])

ent7 = clinical_merged[clinical_merged.enttype==7] #ent7 is health promotion for alcohol
#select only data at least 6 months before diagnosis
ent7 = pd.merge(ent7, cohort[['e_patid', 'dx_minus6m']], how="left", on="e_patid").copy(deep=True)
ent7 = ent7[ent7["eventdate"] < ent7["dx_minus6m"]].copy(deep=True)

curr_drink_clin = getlatest(curr_drink_clin)
curr_drink_clin.rename(columns= {'eventdate':'curr_drink_clin_1'}, inplace = True)
former_drink_clin = getlatest(former_drink_clin)
former_drink_clin.rename(columns= {'eventdate':'former_drink_clin_1'}, inplace = True)
never_drink_clin = getlatest(never_drink_clin)
never_drink_clin.rename(columns= {'eventdate':'never_drink_clin_1'}, inplace = True)
heavy_drink_clin = getlatest(heavy_drink_clin)
heavy_drink_clin.rename(columns= {'eventdate':'heavy_drink_clin_1'}, inplace = True)

curr_drinker = getlatest(curr_drinker)
curr_drinker.rename(columns= {'eventdate':'curr_drinker_1'}, inplace = True)
heavy_drinker = getlatest(heavy_drinker)
heavy_drinker.rename(columns= {'eventdate':'heavy_drinker_1'}, inplace = True)
non_drinker = getlatest(non_drinker)
non_drinker.rename(columns= {'eventdate':'non_drinker_1'}, inplace = True)
ex_drinker = getlatest(ex_drinker)
ex_drinker.rename(columns = {'eventdate':'ex_drinker_1'}, inplace = True)

#get the latest occurence of the heavy drinking flag from the audit and fast tests
heavy372 = getlatest(heavy372)
heavy372.rename(columns= {'eventdate':'heavy372_1'}, inplace = True)

#get the latest occurrence of health promotion for drinking
ent7 = getlatest(ent7)
ent7.rename(columns= {'eventdate':'ent7_1'}, inplace = True)

#merge all the files with data on drinking
dfs_drinking = [curr_drink_clin, former_drink_clin, never_drink_clin, heavy_drink_clin, curr_drinker, heavy_drinker,
                non_drinker, ex_drinker, heavy372, ent7]
all_drinking_data = reduce(lambda left,right: pd.merge(left, right, on=["e_patid"], how="outer"), dfs_drinking) #merge all into one file keeping all rows

#function to define drinking status
all_drinking_data["rec_drink"] = all_drinking_data[["curr_drink_clin_1", "former_drink_clin_1", "never_drink_clin_1", "heavy_drink_clin_1", "curr_drinker_1", "heavy_drinker_1",
                                                    "non_drinker_1", "ex_drinker_1", "heavy372_1", "ent7_1"]].max(axis=1)
#merge in dates 6m and 5y before diagnosis
all_drinking_data = pd.merge(all_drinking_data, cohort[['e_patid', 'dx_minus6m', 'dx_minus5y']], on = "e_patid", how="left")

#function to define drinking status as current, former or never
#prioritise data from ent5 for the definition
#then use data from codes in the clinical files
#then use data from  ent7 (health promotion)

def drinking_status_6m(df):
    global recent_drinking 
    recent_drinking = None
    if((df['rec_drink'] == df['non_drinker_1']) and ((pd.isna(df["curr_drinker_1"])) and(pd.isna(df["ex_drinker_1"])))):
        recent_drinking = "Non drinker"
    elif((df['rec_drink'] == df['non_drinker_1']) and ((pd.notna(df["curr_drinker_1"])) or (pd.notna(df["ex_drinker_1"])))):    
        recent_drinking = "Ex drinker"
    elif(df['rec_drink'] == df['curr_drinker_1']):
        recent_drinking = "Current drinker"
    elif(df['rec_drink'] == df['heavy_drinker_1']):
        recent_drinking = "Current drinker"
    elif(df['rec_drink'] == df['ex_drinker_1']):
        recent_drinking = "Ex drinker"        
    elif(df['rec_drink'] == df['curr_drink_clin_1']):
        recent_drinking = "Current drinker"
    elif(df['rec_drink'] == df['former_drink_clin_1']):
        recent_drinking = "Ex drinker"
    elif(df['rec_drink'] == df['never_drink_clin_1']):
        recent_drinking = "Non drinker"
    else:
        recent_drinking = None
    return recent_drinking

all_drinking_data["drinking_status"]= all_drinking_data.apply(drinking_status_6m, axis=1) 

In [33]:
all_drinking_data.head()

,e_patid,curr_drink_clin_1,former_drink_clin_1,never_drink_clin_1,heavy_drink_clin_1,curr_drinker_1,heavy_drinker_1,non_drinker_1,ex_drinker_1,heavy372_1,ent7_1,rec_drink,dx_minus6m,dx_minus5y,drinking_status
0,4242,2006-08-03,NaT,NaT,NaT,2006-08-03,NaT,NaT,NaT,NaT,2003-01-22,2006-08-03,2007-11-08,2003-05-10,Current drinker
1,8370,2004-11-05,NaT,NaT,NaT,2004-11-05,NaT,NaT,NaT,NaT,NaT,2004-11-05,2005-01-16,2000-07-18,Current drinker
2,13354,2006-06-09,2007-07-06,NaT,2007-07-06,2006-06-09,2003-12-19,NaT,2007-07-06,NaT,NaT,2007-07-06,2008-05-13,2003-11-13,Ex drinker
3,19457,2009-03-19,2005-07-14,NaT,2005-07-14,2009-03-19,2009-03-19,NaT,2005-07-14,NaT,2008-03-06,2009-03-19,2010-02-24,2005-08-26,Current drinker
4,19799,1992-12-31,NaT,NaT,NaT,1992-12-31,NaT,NaT,NaT,NaT,NaT,1992-12-31,2008-11-25,2004-05-27,Current drinker


In [34]:
cohort3 = cohort2.merge(right = all_drinking_data[['e_patid', 'drinking_status']], on = ['e_patid'], how = 'left')
cohort3.head()

,e_patid,first_diagnosis_date,first_diagnosed_in_primary_care,gender,age_at_dx,e_pracid,region,uts,e2015_imd_5,dx_minus6m,dx_minus12m,dx_minus18m,dx_minus24m,dx_minus3y,dx_minus5y,late_dx,Ethnicity,drinking_status
0,24117254,2005-10-04,1.0,1.0,70.0,54.0,10.0,2000-01-12,4.0,2005-04-04,2004-10-04,2004-04-04,2003-10-05,2002-10-05,2000-10-04,0.0,White,Current drinker
1,519045131,2004-02-11,0.0,1.0,74.0,131.0,2.0,1992-12-21,5.0,2003-08-12,2003-02-11,2002-08-12,2002-02-11,2001-02-11,1999-02-11,1.0,White,Current drinker
2,734003216,2009-08-26,1.0,2.0,93.0,216.0,10.0,2003-02-14,4.0,2009-02-24,2008-08-26,2008-02-25,2007-08-27,2006-08-27,2004-08-26,0.0,White,NaN
3,10485798,2014-01-02,0.0,1.0,89.0,98.0,6.0,1990-01-03,4.0,2013-07-03,2013-01-02,2012-07-03,2012-01-03,2011-01-03,2009-01-02,1.0,White,Ex drinker
4,630194220,2003-05-15,0.0,2.0,74.0,220.0,7.0,1997-09-23,5.0,2002-11-13,2002-05-15,2001-11-13,2001-05-15,2000-05-15,1998-05-15,1.0,White,Current drinker


In [35]:
# get lates weight category med codes for each patient
clin_weight = clinical_merged[clinical_merged.medcode.isin(weight_codes.medcode)][['e_patid', 'eventdate', 'medcode']]
ref_weight = referral_merged[referral_merged.medcode.isin(weight_codes.medcode)][['e_patid', 'eventdate', 'medcode']]

weight_combined = pd.concat([clin_weight, ref_weight])
weight_combined = weight_combined.merge(cohort[['e_patid', 'dx_minus6m']], how="left", on="e_patid").copy(deep=True)
weight_combined = weight_combined[weight_combined['eventdate'] < weight_combined['dx_minus6m']]

dates_weight_latest = getlatest(weight_combined[['e_patid', 'eventdate']])

latest_weight_medcodes = dates_weight_latest.merge(weight_combined, how="inner", on=["e_patid", "eventdate"])
latest_weight_medcodes = latest_weight_medcodes.drop_duplicates()

latest_weight_categories = latest_weight_medcodes.merge(weight_codes[['medcode', 'category']], how='left', on='medcode').copy(deep=True)
latest_weight_categories = latest_weight_categories[['e_patid', 'category']]
latest_weight_categories.rename(columns= {'category':'weight_category'}, inplace = True)
latest_weight_categories = latest_weight_categories.drop_duplicates()

latest_weight_categories.head()

,e_patid,weight_category
0,6902,Obese
1,39691,Obese
2,51063,Obese
3,52532,Obese
4,140013,Obese


In [36]:
len(weight_combined.e_patid.unique())

1983

In [37]:
latest_weight_categories.shape[0]

2010

In [38]:
latest_weight_categories['e_patid'].value_counts()

228731221    2
400075221    2
15922758     2
61984858     2
682112221    2
            ..
62053460     1
62040253     1
61986553     1
61901495     1
799557257    1
Name: e_patid, Length: 1983, dtype: int64

In [39]:
latest_weight_categories[latest_weight_categories['e_patid'].isin([23273158, 224480221, 132647221, 1683977, 510375155])]

,e_patid,weight_category
29,1683977,Overweight
30,1683977,Obese
278,23273158,Obese
279,23273158,Overweight
954,132647221,Obese
955,132647221,Overweight
1105,224480221,Obese
1106,224480221,Overweight
1570,510375155,Obese
1571,510375155,Overweight


In [40]:
# if multiple select only obese as duplicates follow this pattern of having both obese and overweight
for i in list(latest_weight_categories.e_patid.unique()):
    if len(latest_weight_categories[latest_weight_categories.e_patid == i]) > 1:
        dfi = latest_weight_categories[latest_weight_categories.e_patid == i]
        index = dfi[dfi.weight_category == 'Overweight'].index
        latest_weight_categories = latest_weight_categories.drop(index)

In [41]:
latest_weight_categories.shape[0]

1983

In [42]:
latest_weight_categories[latest_weight_categories['e_patid'].isin([23273158, 224480221, 132647221, 1683977, 510375155])]

,e_patid,weight_category
30,1683977,Obese
278,23273158,Obese
954,132647221,Obese
1105,224480221,Obese
1570,510375155,Obese


In [43]:
cohort4 = cohort3.merge(right = latest_weight_categories[['e_patid', 'weight_category']], on = ['e_patid'], how = 'left')
cohort4.head()

,e_patid,first_diagnosis_date,first_diagnosed_in_primary_care,gender,age_at_dx,e_pracid,region,uts,e2015_imd_5,dx_minus6m,dx_minus12m,dx_minus18m,dx_minus24m,dx_minus3y,dx_minus5y,late_dx,Ethnicity,drinking_status,weight_category
0,24117254,2005-10-04,1.0,1.0,70.0,54.0,10.0,2000-01-12,4.0,2005-04-04,2004-10-04,2004-04-04,2003-10-05,2002-10-05,2000-10-04,0.0,White,Current drinker,NaN
1,519045131,2004-02-11,0.0,1.0,74.0,131.0,2.0,1992-12-21,5.0,2003-08-12,2003-02-11,2002-08-12,2002-02-11,2001-02-11,1999-02-11,1.0,White,Current drinker,NaN
2,734003216,2009-08-26,1.0,2.0,93.0,216.0,10.0,2003-02-14,4.0,2009-02-24,2008-08-26,2008-02-25,2007-08-27,2006-08-27,2004-08-26,0.0,White,NaN,NaN
3,10485798,2014-01-02,0.0,1.0,89.0,98.0,6.0,1990-01-03,4.0,2013-07-03,2013-01-02,2012-07-03,2012-01-03,2011-01-03,2009-01-02,1.0,White,Ex drinker,NaN
4,630194220,2003-05-15,0.0,2.0,74.0,220.0,7.0,1997-09-23,5.0,2002-11-13,2002-05-15,2001-11-13,2001-05-15,2000-05-15,1998-05-15,1.0,White,Current drinker,NaN


In [44]:
# add anxiety and depression in last 3 years (as can improve) as one variable (as codes crossover)
# also remove depression codes that come with a dementia diagnosis
anxiety_and_depression_codes = set(anxiety_codes.medcode).union(depression_codes.medcode) - set(dementia_codes.medcode)
clin_anxiety_and_depression = clinical_merged[clinical_merged.medcode.isin(anxiety_and_depression_codes)][['e_patid', 'eventdate', 'medcode']]
ref_anxiety_and_depression = referral_merged[referral_merged.medcode.isin(anxiety_and_depression_codes)][['e_patid', 'eventdate', 'medcode']]
test_anxiety_and_depression = test_merged[test_merged.medcode.isin(anxiety_and_depression_codes)][['e_patid', 'eventdate', 'medcode']]

anxiety_and_depression_combined = pd.concat([clin_anxiety_and_depression, ref_anxiety_and_depression, test_anxiety_and_depression])
anxiety_and_depression_combined = anxiety_and_depression_combined.merge(cohort[['e_patid', 'dx_minus6m', 'dx_minus3y']], how="left", on="e_patid").copy(deep=True)
anxiety_and_depression_combined = anxiety_and_depression_combined[anxiety_and_depression_combined['eventdate'] < anxiety_and_depression_combined['dx_minus6m']]
anxiety_and_depression_combined = anxiety_and_depression_combined[anxiety_and_depression_combined['eventdate'] > anxiety_and_depression_combined['dx_minus3y']]

anx_or_depr = []
for i in list(cohort4.e_patid):
    if i in anxiety_and_depression_combined.e_patid.unique():
        anx_or_depr.append(1.0)
    else:
        anx_or_depr.append(0.0)

cohort5 = cohort4.copy(deep=True)
cohort5['anxiety_or_depression_in_last_3y'] = anx_or_depr
cohort5.head()

,e_patid,first_diagnosis_date,first_diagnosed_in_primary_care,gender,age_at_dx,e_pracid,region,uts,e2015_imd_5,dx_minus6m,dx_minus12m,dx_minus18m,dx_minus24m,dx_minus3y,dx_minus5y,late_dx,Ethnicity,drinking_status,weight_category,anxiety_or_depression_in_last_3y
0,24117254,2005-10-04,1.0,1.0,70.0,54.0,10.0,2000-01-12,4.0,2005-04-04,2004-10-04,2004-04-04,2003-10-05,2002-10-05,2000-10-04,0.0,White,Current drinker,NaN,0.0
1,519045131,2004-02-11,0.0,1.0,74.0,131.0,2.0,1992-12-21,5.0,2003-08-12,2003-02-11,2002-08-12,2002-02-11,2001-02-11,1999-02-11,1.0,White,Current drinker,NaN,0.0
2,734003216,2009-08-26,1.0,2.0,93.0,216.0,10.0,2003-02-14,4.0,2009-02-24,2008-08-26,2008-02-25,2007-08-27,2006-08-27,2004-08-26,0.0,White,NaN,NaN,0.0
3,10485798,2014-01-02,0.0,1.0,89.0,98.0,6.0,1990-01-03,4.0,2013-07-03,2013-01-02,2012-07-03,2012-01-03,2011-01-03,2009-01-02,1.0,White,Ex drinker,NaN,0.0
4,630194220,2003-05-15,0.0,2.0,74.0,220.0,7.0,1997-09-23,5.0,2002-11-13,2002-05-15,2001-11-13,2001-05-15,2000-05-15,1998-05-15,1.0,White,Current drinker,NaN,0.0


In [45]:
cohort5.anxiety_or_depression_in_last_3y.describe()

count    20784.000000
mean         0.091609
std          0.288480
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: anxiety_or_depression_in_last_3y, dtype: float64

In [46]:
# add serious mental illness diagnosed at any time before last 6 months (to be relevant for effect on dx time
# other variables similary discount last 6 months
clin_smi = clinical_merged[clinical_merged.medcode.isin(smi_codes.medcode)][['e_patid', 'eventdate', 'medcode']]
ref_smi = referral_merged[referral_merged.medcode.isin(smi_codes.medcode)][['e_patid', 'eventdate', 'medcode']]
test_smi = test_merged[test_merged.medcode.isin(smi_codes.medcode)][['e_patid', 'eventdate', 'medcode']]

smi_combined = pd.concat([clin_smi, ref_smi, test_smi])
smi_combined = smi_combined.merge(cohort[['e_patid', 'dx_minus6m', 'dx_minus3y']], how="left", on="e_patid").copy(deep=True)
smi_combined = smi_combined[smi_combined['eventdate'] < smi_combined['dx_minus6m']]

smi = []
for i in list(cohort5.e_patid):
    if i in smi_combined.e_patid.unique():
        smi.append(1.0)
    else:
        smi.append(0.0)

cohort6 = cohort5.copy(deep=True)
cohort6['smi'] = smi
cohort6.head()

,e_patid,first_diagnosis_date,first_diagnosed_in_primary_care,gender,age_at_dx,e_pracid,region,uts,e2015_imd_5,dx_minus6m,dx_minus12m,dx_minus18m,dx_minus24m,dx_minus3y,dx_minus5y,late_dx,Ethnicity,drinking_status,weight_category,anxiety_or_depression_in_last_3y,smi
0,24117254,2005-10-04,1.0,1.0,70.0,54.0,10.0,2000-01-12,4.0,2005-04-04,2004-10-04,2004-04-04,2003-10-05,2002-10-05,2000-10-04,0.0,White,Current drinker,NaN,0.0,0.0
1,519045131,2004-02-11,0.0,1.0,74.0,131.0,2.0,1992-12-21,5.0,2003-08-12,2003-02-11,2002-08-12,2002-02-11,2001-02-11,1999-02-11,1.0,White,Current drinker,NaN,0.0,0.0
2,734003216,2009-08-26,1.0,2.0,93.0,216.0,10.0,2003-02-14,4.0,2009-02-24,2008-08-26,2008-02-25,2007-08-27,2006-08-27,2004-08-26,0.0,White,NaN,NaN,0.0,0.0
3,10485798,2014-01-02,0.0,1.0,89.0,98.0,6.0,1990-01-03,4.0,2013-07-03,2013-01-02,2012-07-03,2012-01-03,2011-01-03,2009-01-02,1.0,White,Ex drinker,NaN,0.0,0.0
4,630194220,2003-05-15,0.0,2.0,74.0,220.0,7.0,1997-09-23,5.0,2002-11-13,2002-05-15,2001-11-13,2001-05-15,2000-05-15,1998-05-15,1.0,White,Current drinker,NaN,0.0,0.0


In [47]:
cohort6.smi.describe()

count    20784.000000
mean         0.023287
std          0.150818
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: smi, dtype: float64

In [48]:
# add epilepsy column
clin_epilepsy = clinical_merged[clinical_merged.medcode.isin(epilepsy_codes.medcode)][['e_patid', 'eventdate', 'medcode']]
ref_epilepsy = referral_merged[referral_merged.medcode.isin(epilepsy_codes.medcode)][['e_patid', 'eventdate', 'medcode']]
test_epilepsy = test_merged[test_merged.medcode.isin(epilepsy_codes.medcode)][['e_patid', 'eventdate', 'medcode']]

epilepsy_combined = pd.concat([clin_epilepsy, ref_epilepsy, test_epilepsy])
epilepsy_combined = epilepsy_combined.merge(cohort[['e_patid', 'dx_minus6m', 'dx_minus3y']], how="left", on="e_patid").copy(deep=True)
epilepsy_combined = epilepsy_combined[epilepsy_combined['eventdate'] < epilepsy_combined['dx_minus6m']]

epilepsy = []
for i in list(cohort6.e_patid):
    if i in epilepsy_combined.e_patid.unique():
        epilepsy.append(1.0)
    else:
        epilepsy.append(0.0)

cohort7 = cohort6.copy(deep=True)
cohort7['epilepsy'] = epilepsy
cohort7.head()

,e_patid,first_diagnosis_date,first_diagnosed_in_primary_care,gender,age_at_dx,e_pracid,region,uts,e2015_imd_5,dx_minus6m,dx_minus12m,dx_minus18m,dx_minus24m,dx_minus3y,dx_minus5y,late_dx,Ethnicity,drinking_status,weight_category,anxiety_or_depression_in_last_3y,smi,epilepsy
0,24117254,2005-10-04,1.0,1.0,70.0,54.0,10.0,2000-01-12,4.0,2005-04-04,2004-10-04,2004-04-04,2003-10-05,2002-10-05,2000-10-04,0.0,White,Current drinker,NaN,0.0,0.0,0.0
1,519045131,2004-02-11,0.0,1.0,74.0,131.0,2.0,1992-12-21,5.0,2003-08-12,2003-02-11,2002-08-12,2002-02-11,2001-02-11,1999-02-11,1.0,White,Current drinker,NaN,0.0,0.0,0.0
2,734003216,2009-08-26,1.0,2.0,93.0,216.0,10.0,2003-02-14,4.0,2009-02-24,2008-08-26,2008-02-25,2007-08-27,2006-08-27,2004-08-26,0.0,White,NaN,NaN,0.0,0.0,0.0
3,10485798,2014-01-02,0.0,1.0,89.0,98.0,6.0,1990-01-03,4.0,2013-07-03,2013-01-02,2012-07-03,2012-01-03,2011-01-03,2009-01-02,1.0,White,Ex drinker,NaN,0.0,0.0,0.0
4,630194220,2003-05-15,0.0,2.0,74.0,220.0,7.0,1997-09-23,5.0,2002-11-13,2002-05-15,2001-11-13,2001-05-15,2000-05-15,1998-05-15,1.0,White,Current drinker,NaN,0.0,0.0,0.0


In [49]:
cohort7.epilepsy.describe()

count    20784.000000
mean         0.024634
std          0.155012
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: epilepsy, dtype: float64

In [50]:
# add asthma column
clin_asthma = clinical_merged[clinical_merged.medcode.isin(asthma_codes.medcode)][['e_patid', 'eventdate', 'medcode']]
ref_asthma = referral_merged[referral_merged.medcode.isin(asthma_codes.medcode)][['e_patid', 'eventdate', 'medcode']]
test_asthma = test_merged[test_merged.medcode.isin(asthma_codes.medcode)][['e_patid', 'eventdate', 'medcode']]

asthma_combined = pd.concat([clin_asthma, ref_asthma, test_asthma])
asthma_combined = asthma_combined.merge(cohort[['e_patid', 'dx_minus6m', 'dx_minus3y']], how="left", on="e_patid").copy(deep=True)
asthma_combined = asthma_combined[asthma_combined['eventdate'] < asthma_combined['dx_minus6m']]

asthma = []
for i in list(cohort7.e_patid):
    if i in asthma_combined.e_patid.unique():
        asthma.append(1.0)
    else:
        asthma.append(0.0)

cohort8 = cohort7.copy(deep=True)
cohort8['asthma'] = asthma
cohort8.head()

,e_patid,first_diagnosis_date,first_diagnosed_in_primary_care,gender,age_at_dx,e_pracid,region,uts,e2015_imd_5,dx_minus6m,dx_minus12m,dx_minus18m,dx_minus24m,dx_minus3y,dx_minus5y,late_dx,Ethnicity,drinking_status,weight_category,anxiety_or_depression_in_last_3y,smi,epilepsy,asthma
0,24117254,2005-10-04,1.0,1.0,70.0,54.0,10.0,2000-01-12,4.0,2005-04-04,2004-10-04,2004-04-04,2003-10-05,2002-10-05,2000-10-04,0.0,White,Current drinker,NaN,0.0,0.0,0.0,0.0
1,519045131,2004-02-11,0.0,1.0,74.0,131.0,2.0,1992-12-21,5.0,2003-08-12,2003-02-11,2002-08-12,2002-02-11,2001-02-11,1999-02-11,1.0,White,Current drinker,NaN,0.0,0.0,0.0,0.0
2,734003216,2009-08-26,1.0,2.0,93.0,216.0,10.0,2003-02-14,4.0,2009-02-24,2008-08-26,2008-02-25,2007-08-27,2006-08-27,2004-08-26,0.0,White,NaN,NaN,0.0,0.0,0.0,0.0
3,10485798,2014-01-02,0.0,1.0,89.0,98.0,6.0,1990-01-03,4.0,2013-07-03,2013-01-02,2012-07-03,2012-01-03,2011-01-03,2009-01-02,1.0,White,Ex drinker,NaN,0.0,0.0,0.0,0.0
4,630194220,2003-05-15,0.0,2.0,74.0,220.0,7.0,1997-09-23,5.0,2002-11-13,2002-05-15,2001-11-13,2001-05-15,2000-05-15,1998-05-15,1.0,White,Current drinker,NaN,0.0,0.0,0.0,0.0


In [51]:
cohort8.asthma.describe()

count    20784.000000
mean         0.094400
std          0.292391
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: asthma, dtype: float64

In [52]:
# add chronic fatigue syndrome column
clin_cfs = clinical_merged[clinical_merged.medcode.isin(cfs_codes.medcode)][['e_patid', 'eventdate', 'medcode']]
ref_cfs = referral_merged[referral_merged.medcode.isin(cfs_codes.medcode)][['e_patid', 'eventdate', 'medcode']]
test_cfs = test_merged[test_merged.medcode.isin(cfs_codes.medcode)][['e_patid', 'eventdate', 'medcode']]

cfs_combined = pd.concat([clin_cfs, ref_cfs, test_cfs])
cfs_combined = cfs_combined.merge(cohort[['e_patid', 'dx_minus6m', 'dx_minus3y']], how="left", on="e_patid").copy(deep=True)
cfs_combined = cfs_combined[cfs_combined['eventdate'] < cfs_combined['dx_minus6m']]

cfs = []
for i in list(cohort8.e_patid):
    if i in cfs_combined.e_patid.unique():
        cfs.append(1.0)
    else:
        cfs.append(0.0)

cohort9 = cohort8.copy(deep=True)
cohort9['cfs'] = cfs
cohort9.head()

,e_patid,first_diagnosis_date,first_diagnosed_in_primary_care,gender,age_at_dx,e_pracid,region,uts,e2015_imd_5,dx_minus6m,dx_minus12m,dx_minus18m,dx_minus24m,dx_minus3y,dx_minus5y,late_dx,Ethnicity,drinking_status,weight_category,anxiety_or_depression_in_last_3y,smi,epilepsy,asthma,cfs
0,24117254,2005-10-04,1.0,1.0,70.0,54.0,10.0,2000-01-12,4.0,2005-04-04,2004-10-04,2004-04-04,2003-10-05,2002-10-05,2000-10-04,0.0,White,Current drinker,NaN,0.0,0.0,0.0,0.0,0.0
1,519045131,2004-02-11,0.0,1.0,74.0,131.0,2.0,1992-12-21,5.0,2003-08-12,2003-02-11,2002-08-12,2002-02-11,2001-02-11,1999-02-11,1.0,White,Current drinker,NaN,0.0,0.0,0.0,0.0,0.0
2,734003216,2009-08-26,1.0,2.0,93.0,216.0,10.0,2003-02-14,4.0,2009-02-24,2008-08-26,2008-02-25,2007-08-27,2006-08-27,2004-08-26,0.0,White,NaN,NaN,0.0,0.0,0.0,0.0,0.0
3,10485798,2014-01-02,0.0,1.0,89.0,98.0,6.0,1990-01-03,4.0,2013-07-03,2013-01-02,2012-07-03,2012-01-03,2011-01-03,2009-01-02,1.0,White,Ex drinker,NaN,0.0,0.0,0.0,0.0,0.0
4,630194220,2003-05-15,0.0,2.0,74.0,220.0,7.0,1997-09-23,5.0,2002-11-13,2002-05-15,2001-11-13,2001-05-15,2000-05-15,1998-05-15,1.0,White,Current drinker,NaN,0.0,0.0,0.0,0.0,0.0


In [53]:
cohort9.cfs.describe()

count    20784.000000
mean         0.004186
std          0.064565
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: cfs, dtype: float64

In [54]:
# add chronic kidney disease column
clin_ckd = clinical_merged[clinical_merged.medcode.isin(ckd_codes.medcode)][['e_patid', 'eventdate', 'medcode']]
ref_ckd = referral_merged[referral_merged.medcode.isin(ckd_codes.medcode)][['e_patid', 'eventdate', 'medcode']]
test_ckd = test_merged[test_merged.medcode.isin(ckd_codes.medcode)][['e_patid', 'eventdate', 'medcode']]

ckd_combined = pd.concat([clin_ckd, ref_ckd, test_ckd])
ckd_combined = ckd_combined.merge(cohort[['e_patid', 'dx_minus6m', 'dx_minus3y']], how="left", on="e_patid").copy(deep=True)
ckd_combined = ckd_combined[ckd_combined['eventdate'] < ckd_combined['dx_minus6m']]

ckd = []
for i in list(cohort9.e_patid):
    if i in ckd_combined.e_patid.unique():
        ckd.append(1.0)
    else:
        ckd.append(0.0)

cohort10 = cohort9.copy(deep=True)
cohort10['ckd'] = ckd
cohort10.head()

,e_patid,first_diagnosis_date,first_diagnosed_in_primary_care,gender,age_at_dx,e_pracid,region,uts,e2015_imd_5,dx_minus6m,dx_minus12m,dx_minus18m,dx_minus24m,dx_minus3y,dx_minus5y,late_dx,Ethnicity,drinking_status,weight_category,anxiety_or_depression_in_last_3y,smi,epilepsy,asthma,cfs,ckd
0,24117254,2005-10-04,1.0,1.0,70.0,54.0,10.0,2000-01-12,4.0,2005-04-04,2004-10-04,2004-04-04,2003-10-05,2002-10-05,2000-10-04,0.0,White,Current drinker,NaN,0.0,0.0,0.0,0.0,0.0,0.0
1,519045131,2004-02-11,0.0,1.0,74.0,131.0,2.0,1992-12-21,5.0,2003-08-12,2003-02-11,2002-08-12,2002-02-11,2001-02-11,1999-02-11,1.0,White,Current drinker,NaN,0.0,0.0,0.0,0.0,0.0,0.0
2,734003216,2009-08-26,1.0,2.0,93.0,216.0,10.0,2003-02-14,4.0,2009-02-24,2008-08-26,2008-02-25,2007-08-27,2006-08-27,2004-08-26,0.0,White,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
3,10485798,2014-01-02,0.0,1.0,89.0,98.0,6.0,1990-01-03,4.0,2013-07-03,2013-01-02,2012-07-03,2012-01-03,2011-01-03,2009-01-02,1.0,White,Ex drinker,NaN,0.0,0.0,0.0,0.0,0.0,0.0
4,630194220,2003-05-15,0.0,2.0,74.0,220.0,7.0,1997-09-23,5.0,2002-11-13,2002-05-15,2001-11-13,2001-05-15,2000-05-15,1998-05-15,1.0,White,Current drinker,NaN,0.0,0.0,0.0,0.0,0.0,0.0


In [55]:
cohort10.ckd.describe()

count    20784.000000
mean         0.168351
std          0.374186
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: ckd, dtype: float64

In [56]:
# add chronic obstructive pulmonary disease column
clin_copd = clinical_merged[clinical_merged.medcode.isin(copd_codes.medcode)][['e_patid', 'eventdate', 'medcode']]
ref_copd = referral_merged[referral_merged.medcode.isin(copd_codes.medcode)][['e_patid', 'eventdate', 'medcode']]
test_copd = test_merged[test_merged.medcode.isin(copd_codes.medcode)][['e_patid', 'eventdate', 'medcode']]

copd_combined = pd.concat([clin_copd, ref_copd, test_copd])
copd_combined = copd_combined.merge(cohort[['e_patid', 'dx_minus6m', 'dx_minus3y']], how="left", on="e_patid").copy(deep=True)
copd_combined = copd_combined[copd_combined['eventdate'] < copd_combined['dx_minus6m']]

copd = []
for i in list(cohort10.e_patid):
    if i in copd_combined.e_patid.unique():
        copd.append(1.0)
    else:
        copd.append(0.0)

cohort11 = cohort10.copy(deep=True)
cohort11['copd'] = copd
cohort11.head()

,e_patid,first_diagnosis_date,first_diagnosed_in_primary_care,gender,age_at_dx,e_pracid,region,uts,e2015_imd_5,dx_minus6m,dx_minus12m,dx_minus18m,dx_minus24m,dx_minus3y,dx_minus5y,late_dx,Ethnicity,drinking_status,weight_category,anxiety_or_depression_in_last_3y,smi,epilepsy,asthma,cfs,ckd,copd
0,24117254,2005-10-04,1.0,1.0,70.0,54.0,10.0,2000-01-12,4.0,2005-04-04,2004-10-04,2004-04-04,2003-10-05,2002-10-05,2000-10-04,0.0,White,Current drinker,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,519045131,2004-02-11,0.0,1.0,74.0,131.0,2.0,1992-12-21,5.0,2003-08-12,2003-02-11,2002-08-12,2002-02-11,2001-02-11,1999-02-11,1.0,White,Current drinker,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,734003216,2009-08-26,1.0,2.0,93.0,216.0,10.0,2003-02-14,4.0,2009-02-24,2008-08-26,2008-02-25,2007-08-27,2006-08-27,2004-08-26,0.0,White,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10485798,2014-01-02,0.0,1.0,89.0,98.0,6.0,1990-01-03,4.0,2013-07-03,2013-01-02,2012-07-03,2012-01-03,2011-01-03,2009-01-02,1.0,White,Ex drinker,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,630194220,2003-05-15,0.0,2.0,74.0,220.0,7.0,1997-09-23,5.0,2002-11-13,2002-05-15,2001-11-13,2001-05-15,2000-05-15,1998-05-15,1.0,White,Current drinker,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [57]:
cohort11.copd.describe()

count    20784.000000
mean         0.056486
std          0.230863
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: copd, dtype: float64

In [58]:
# add hypertension column
clin_hypertension = clinical_merged[clinical_merged.medcode.isin(hypertension_codes.medcode)][['e_patid', 'eventdate', 'medcode']]
ref_hypertension = referral_merged[referral_merged.medcode.isin(hypertension_codes.medcode)][['e_patid', 'eventdate', 'medcode']]
test_hypertension = test_merged[test_merged.medcode.isin(hypertension_codes.medcode)][['e_patid', 'eventdate', 'medcode']]

hypertension_combined = pd.concat([clin_hypertension, ref_hypertension, test_hypertension])
hypertension_combined = hypertension_combined.merge(cohort[['e_patid', 'dx_minus6m', 'dx_minus3y']], how="left", on="e_patid").copy(deep=True)
hypertension_combined = hypertension_combined[hypertension_combined['eventdate'] < hypertension_combined['dx_minus6m']]

hypertension = []
for i in list(cohort11.e_patid):
    if i in hypertension_combined.e_patid.unique():
        hypertension.append(1.0)
    else:
        hypertension.append(0.0)

cohort12 = cohort11.copy(deep=True)
cohort12['hypertension'] = hypertension
cohort12.head()

,e_patid,first_diagnosis_date,first_diagnosed_in_primary_care,gender,age_at_dx,e_pracid,region,uts,e2015_imd_5,dx_minus6m,dx_minus12m,dx_minus18m,dx_minus24m,dx_minus3y,dx_minus5y,late_dx,Ethnicity,drinking_status,weight_category,anxiety_or_depression_in_last_3y,smi,epilepsy,asthma,cfs,ckd,copd,hypertension
0,24117254,2005-10-04,1.0,1.0,70.0,54.0,10.0,2000-01-12,4.0,2005-04-04,2004-10-04,2004-04-04,2003-10-05,2002-10-05,2000-10-04,0.0,White,Current drinker,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,519045131,2004-02-11,0.0,1.0,74.0,131.0,2.0,1992-12-21,5.0,2003-08-12,2003-02-11,2002-08-12,2002-02-11,2001-02-11,1999-02-11,1.0,White,Current drinker,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,734003216,2009-08-26,1.0,2.0,93.0,216.0,10.0,2003-02-14,4.0,2009-02-24,2008-08-26,2008-02-25,2007-08-27,2006-08-27,2004-08-26,0.0,White,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,10485798,2014-01-02,0.0,1.0,89.0,98.0,6.0,1990-01-03,4.0,2013-07-03,2013-01-02,2012-07-03,2012-01-03,2011-01-03,2009-01-02,1.0,White,Ex drinker,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,630194220,2003-05-15,0.0,2.0,74.0,220.0,7.0,1997-09-23,5.0,2002-11-13,2002-05-15,2001-11-13,2001-05-15,2000-05-15,1998-05-15,1.0,White,Current drinker,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [59]:
cohort12.hypertension.describe()

count    20784.000000
mean         0.506062
std          0.499975
min          0.000000
25%          0.000000
50%          1.000000
75%          1.000000
max          1.000000
Name: hypertension, dtype: float64

In [60]:
# add osteoporosis column
clin_osteoporosis = clinical_merged[clinical_merged.medcode.isin(osteoporosis_codes.medcode)][['e_patid', 'eventdate', 'medcode']]
ref_osteoporosis = referral_merged[referral_merged.medcode.isin(osteoporosis_codes.medcode)][['e_patid', 'eventdate', 'medcode']]
test_osteoporosis = test_merged[test_merged.medcode.isin(osteoporosis_codes.medcode)][['e_patid', 'eventdate', 'medcode']]

osteoporosis_combined = pd.concat([clin_osteoporosis, ref_osteoporosis, test_osteoporosis])
osteoporosis_combined = osteoporosis_combined.merge(cohort[['e_patid', 'dx_minus6m', 'dx_minus3y']], how="left", on="e_patid").copy(deep=True)
osteoporosis_combined = osteoporosis_combined[osteoporosis_combined['eventdate'] < osteoporosis_combined['dx_minus6m']]

osteoporosis = []
for i in list(cohort12.e_patid):
    if i in osteoporosis_combined.e_patid.unique():
        osteoporosis.append(1.0)
    else:
        osteoporosis.append(0.0)

cohort13 = cohort12.copy(deep=True)
cohort13['osteoporosis'] = osteoporosis
cohort13.head()

,e_patid,first_diagnosis_date,first_diagnosed_in_primary_care,gender,age_at_dx,e_pracid,region,uts,e2015_imd_5,dx_minus6m,dx_minus12m,dx_minus18m,dx_minus24m,dx_minus3y,dx_minus5y,late_dx,Ethnicity,drinking_status,weight_category,anxiety_or_depression_in_last_3y,smi,epilepsy,asthma,cfs,ckd,copd,hypertension,osteoporosis
0,24117254,2005-10-04,1.0,1.0,70.0,54.0,10.0,2000-01-12,4.0,2005-04-04,2004-10-04,2004-04-04,2003-10-05,2002-10-05,2000-10-04,0.0,White,Current drinker,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,519045131,2004-02-11,0.0,1.0,74.0,131.0,2.0,1992-12-21,5.0,2003-08-12,2003-02-11,2002-08-12,2002-02-11,2001-02-11,1999-02-11,1.0,White,Current drinker,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,734003216,2009-08-26,1.0,2.0,93.0,216.0,10.0,2003-02-14,4.0,2009-02-24,2008-08-26,2008-02-25,2007-08-27,2006-08-27,2004-08-26,0.0,White,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,10485798,2014-01-02,0.0,1.0,89.0,98.0,6.0,1990-01-03,4.0,2013-07-03,2013-01-02,2012-07-03,2012-01-03,2011-01-03,2009-01-02,1.0,White,Ex drinker,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,630194220,2003-05-15,0.0,2.0,74.0,220.0,7.0,1997-09-23,5.0,2002-11-13,2002-05-15,2001-11-13,2001-05-15,2000-05-15,1998-05-15,1.0,White,Current drinker,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [61]:
cohort13.osteoporosis.describe()

count    20784.000000
mean         0.096228
std          0.294911
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: osteoporosis, dtype: float64

In [62]:
# add chronic heart disease column
clin_chd = clinical_merged[clinical_merged.medcode.isin(chd_codes.medcode)][['e_patid', 'eventdate', 'medcode']]
ref_chd = referral_merged[referral_merged.medcode.isin(chd_codes.medcode)][['e_patid', 'eventdate', 'medcode']]
test_chd = test_merged[test_merged.medcode.isin(chd_codes.medcode)][['e_patid', 'eventdate', 'medcode']]

chd_combined = pd.concat([clin_chd, ref_chd, test_chd])
chd_combined = chd_combined.merge(cohort[['e_patid', 'dx_minus6m', 'dx_minus3y']], how="left", on="e_patid").copy(deep=True)
chd_combined = chd_combined[chd_combined['eventdate'] < chd_combined['dx_minus6m']]

chd = []
for i in list(cohort13.e_patid):
    if i in chd_combined.e_patid.unique():
        chd.append(1.0)
    else:
        chd.append(0.0)

cohort14 = cohort13.copy(deep=True)
cohort14['chd'] = chd
cohort14.head()

,e_patid,first_diagnosis_date,first_diagnosed_in_primary_care,gender,age_at_dx,e_pracid,region,uts,e2015_imd_5,dx_minus6m,dx_minus12m,dx_minus18m,dx_minus24m,dx_minus3y,dx_minus5y,late_dx,Ethnicity,drinking_status,weight_category,anxiety_or_depression_in_last_3y,smi,epilepsy,asthma,cfs,ckd,copd,hypertension,osteoporosis,chd
0,24117254,2005-10-04,1.0,1.0,70.0,54.0,10.0,2000-01-12,4.0,2005-04-04,2004-10-04,2004-04-04,2003-10-05,2002-10-05,2000-10-04,0.0,White,Current drinker,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,519045131,2004-02-11,0.0,1.0,74.0,131.0,2.0,1992-12-21,5.0,2003-08-12,2003-02-11,2002-08-12,2002-02-11,2001-02-11,1999-02-11,1.0,White,Current drinker,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,734003216,2009-08-26,1.0,2.0,93.0,216.0,10.0,2003-02-14,4.0,2009-02-24,2008-08-26,2008-02-25,2007-08-27,2006-08-27,2004-08-26,0.0,White,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,10485798,2014-01-02,0.0,1.0,89.0,98.0,6.0,1990-01-03,4.0,2013-07-03,2013-01-02,2012-07-03,2012-01-03,2011-01-03,2009-01-02,1.0,White,Ex drinker,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,630194220,2003-05-15,0.0,2.0,74.0,220.0,7.0,1997-09-23,5.0,2002-11-13,2002-05-15,2001-11-13,2001-05-15,2000-05-15,1998-05-15,1.0,White,Current drinker,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [63]:
cohort14.chd.describe()

count    20784.000000
mean         0.174942
std          0.379927
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: chd, dtype: float64

In [64]:
# add heart failure column
clin_heart_failure = clinical_merged[clinical_merged.medcode.isin(heart_failure_codes.medcode)][['e_patid', 'eventdate', 'medcode']]
ref_heart_failure = referral_merged[referral_merged.medcode.isin(heart_failure_codes.medcode)][['e_patid', 'eventdate', 'medcode']]
test_heart_failure = test_merged[test_merged.medcode.isin(heart_failure_codes.medcode)][['e_patid', 'eventdate', 'medcode']]

heart_failure_combined = pd.concat([clin_heart_failure, ref_heart_failure, test_heart_failure])
heart_failure_combined = heart_failure_combined.merge(cohort[['e_patid', 'dx_minus6m', 'dx_minus3y']], how="left", on="e_patid").copy(deep=True)
heart_failure_combined = heart_failure_combined[heart_failure_combined['eventdate'] < heart_failure_combined['dx_minus6m']]

heart_failure = []
for i in list(cohort14.e_patid):
    if i in heart_failure_combined.e_patid.unique():
        heart_failure.append(1.0)
    else:
        heart_failure.append(0.0)

cohort15 = cohort14.copy(deep=True)
cohort15['heart_failure'] = heart_failure
cohort15.head()

,e_patid,first_diagnosis_date,first_diagnosed_in_primary_care,gender,age_at_dx,e_pracid,region,uts,e2015_imd_5,dx_minus6m,dx_minus12m,dx_minus18m,dx_minus24m,dx_minus3y,dx_minus5y,late_dx,Ethnicity,drinking_status,weight_category,anxiety_or_depression_in_last_3y,smi,epilepsy,asthma,cfs,ckd,copd,hypertension,osteoporosis,chd,heart_failure
0,24117254,2005-10-04,1.0,1.0,70.0,54.0,10.0,2000-01-12,4.0,2005-04-04,2004-10-04,2004-04-04,2003-10-05,2002-10-05,2000-10-04,0.0,White,Current drinker,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,519045131,2004-02-11,0.0,1.0,74.0,131.0,2.0,1992-12-21,5.0,2003-08-12,2003-02-11,2002-08-12,2002-02-11,2001-02-11,1999-02-11,1.0,White,Current drinker,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,734003216,2009-08-26,1.0,2.0,93.0,216.0,10.0,2003-02-14,4.0,2009-02-24,2008-08-26,2008-02-25,2007-08-27,2006-08-27,2004-08-26,0.0,White,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,10485798,2014-01-02,0.0,1.0,89.0,98.0,6.0,1990-01-03,4.0,2013-07-03,2013-01-02,2012-07-03,2012-01-03,2011-01-03,2009-01-02,1.0,White,Ex drinker,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,630194220,2003-05-15,0.0,2.0,74.0,220.0,7.0,1997-09-23,5.0,2002-11-13,2002-05-15,2001-11-13,2001-05-15,2000-05-15,1998-05-15,1.0,White,Current drinker,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [65]:
cohort15.heart_failure.describe()

count    20784.000000
mean         0.065002
std          0.246535
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: heart_failure, dtype: float64

In [66]:
# save to parquet
# note all comorbidity columns are one-hot categoricals
cohort15.to_parquet(os.getcwd() + '/cohort_with_risk_factors.parquet')